In [1]:
%load_ext autoreload
%autoreload 2
import nmslib

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [3]:
import numpy as np

# create a random matrix to index
data = np.random.randn(9_619_086, 3).astype(np.float32)
query = np.random.randn(256**3, 3).astype(np.float32)

In [4]:
# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='hnsw', space='l2')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

# query for the nearest neighbours of the first datapoint
# ids, distances = index.knnQuery(query[0], k=1)

# get all nearest neighbours for all the datapoint
# using a pool of 4 threads to compute
neighbours = index.knnQueryBatch(query, k=1, num_threads=20) # put query data here


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
******************************************************


In [21]:
ids

array([124763], dtype=int32)

In [13]:
np.shape(neighbours)

(10000, 2, 1)